<a href="https://colab.research.google.com/github/gayoooon1/2021-RecSys/blob/main/Variational_Autoencoders_for_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



https://arxiv.org/pdf/1802.05814.pdf  논문 보고, 코드 살표보기   

코드 실행해보는 이유 (목표)   
1. raw data에서 preprocessing 실행해서 데이터셋 구축
2. VAE가 어떻게 추천시스템에서 동작을 하는지   
  - 학습
    * Bag of words로 된 xu input(다항분포로 된)
    * Sample a batch of users
    * ELBO 최소화하는 파이와 세타 찾기
      * 가우시안을 따르는 q(zu)를 통해 p(z|x)를 잘 만들어낼 수 있는 이상함수 파라미터 찾는 것과 이상적 z를 찾는 것이 포함 (이 때 mu,sigma u는 data dependent, sampling을 거침 - 병목현상 방지 + reparameterization trick)
  - 예측
    *   f theta(z)를 통해 unnormalized multinomial probability rank를 매긴 뒤 softmax로 normalize를 해준다.
    * 전체 아이템 I에 대한 확률분포를 얻는다. (`Mult-VAE`는 z=m pi(x) 즉 평균, `Mult-DAE` z=single value)
3. 어떻게 모델 평가를 하는지 from 논문 section 4.2에서 모델 평가 방법

# Abstract   
* VAE를 사용함으로써 Collaborative Filtering 기반의 선형 모델들이 가지는 표현에 대한 한계점 개선.    

* 다른 분야와 달리 추천시스템에서 활용되지 않았던 mulitinomial distribution기반의 generative모델을 활용하고 파라미터 추정을 위해 베이지안 추론을 사용.   
  → 다항분포 기반 (확률실험의 결과로 k개의 가능한 경우가 발생할 때 나타나는 분포)

* 목적함수를 학습하는데 있어서 기존 VAE의 수식과 다르게 regularization parameter를 사용하였는데 anneling 기법을 통해 파라미터를 효과적으로 튜닝하여 성능을 상승. ❓❓❓ 

# 1. Introduction   
잠재 요인 모델은 본질적으로 선형이기 때문에 모델 용량을 제한한다는 문제가 있다. 이전 연구는 선형 잠재 요인 모델에 조작된 비선형 피쳐들을 추가하는 것이 추천 성능을 높인다고 제시하였다. 

사용자들에 의한 sparse 신호들을 활용하고 오버피팅을 피하기 위해서, 이 논문에서는 사용자와 아이템 사이들의 확률적 strength를 공유하는 확률적 잠재 변수 모델을 만들었다. 학제적으로 기초적인 베이시안 접근이 데이터의 부족과 상관없이 더욱 나은 성능을 보였다.  


추천시스템에서는 mean average precision and normalized
discounted cumulative gain와 같은 랭킹 기반의 측정방식으로 평가되는데, TOP-N ranking loss는 최적화 되기 어려우며 direct ranking loss는 보통 근사된다. 이에 따라 우리는 multinomial likelihoods가 implicit feedback 데이터에서 잘 적합되고 Gaussian과 logistic과 같은 likelihood function보다 ranking loss를 최적화를 잘 할 수 있다.


https://pyy0715.github.io/VAE_CF/   
https://github.com/dawenl/vae_cf/blob/master/VAE_ML20M_WWW2018.ipynb   

https://github.com/pyy0715/vae-cf-pyorch/

# 2. Method

* We use u ∈ {1, . . . ,U } to index users and i ∈ {1, . . . , I } to index
items. In this work, we consider learning with implicit feedback. (암묵적 피드백 (클릭수)로 학습)   
* The user-by-item interaction matrix is the click matrix
X ∈ N
U ×I
.    
* The lower case xu = [xu1, . . . , xu I]
⊤ ∈ N
I
is a bag -of words vector with the number of clicks for each item from user u. (유저 u에 대해 각 아이템에 대한 클릭 수를 bag of words 형태 벡터로 나타낼 수 있음)

## 2.1 Model   
Generative 과정은 모든 유저 u에 대해서 k차원을 가지는 latent representation zu에서 샘플링 되고 zu는 prior로 Standard Gaussian 분포를 가진다. = generator 학습할 때 z 샘플 잘 만드는 이상적인 sampling 함수, 그니까 z는 prior에서 샘플링해서 얻어내는 것임 우리가 다루기 쉬운 분포로   

**zu는 non-linear function f theta 를 통과하여 전체 아이템에 대해서 확률 분포를 생성할 수 있다. f theta 함수는 파라미터 theta를 가지는 multilayer perceptron, 통과하면 전체 아이템에 대한 확률 벡터 pi(zu) 를 구하기 위해서 softmax를 이용해 normalize를 수행한다.**

## 2.2 Variational Inference   

근사하는 과정 !! 데이터셋에서 유저와 아이템의 수가 커질수록 optimize를 해야하는 m,sigma의 파라미터 수도 점점 많아진다. 즉 이는 수백만명의 유저와 아이템이 있는 추천시스템에서는 병목 현상(bottleneck)이 발생할 수 있기 때문에 VAE에서는 data-dependent한 함수를 통하여 파라미터를 추정한다.   

### 2.2.2 Alternative interpretation of elbo   

The objective of Multi-VAE^{PR} (evidence lower-bound, or ELBO) for a single user $u$ is:$$
\mathcal{L}_u(\theta, \phi) = \mathbb{E}_{q_\phi(z_u | x_u)}[\log p_\theta(x_u | z_u)] - \beta \cdot KL(q_\phi(z_u | x_u) \| p(z_u))
$$where $q_\phi$ is the approximating variational distribution (inference model). 

ELBO의 첫 번째 식은 샘플링 함수에 대한 negative log likelihood인 reconstruction error, posterior와 prior를 유사하도록 만드는 **KL term은 regularization으로 볼 수 있다. 이는 trade-off 관계, 논문에서는 이러한 regularization를 조절할 수 있는 파라미터를 도입해서 ELBO를 확장**   

❓❓❓ **파라미터 $\beta$는 왜 나온걸까...? 정규화의 의미**   

heuristic하게 0부터 시작하여 1까지 점진적으로 증가시키면서 $\beta$를 탐색하는 과정 =  KL annealing   

=> multinomial likelihood기반의 partially regularized 되어진 VAE를 `Mult-VAE`라고 함

## 2.3 A taxonomy of autoencoders   
AE와 DAE는 지금까지의 VAE와는 구분되는 특징 :   
DAE는 VAE에서 prior를 근사하는 variational distribution 
를 optimize, input layer마다 dropout를 적용함으로써 overfitting을 방지하고 성능을 개선   
→ 따라서 Mult-VAE와 함께 multinomial likelihood 기반의 denoising autoencoder도 Mult-DAE 같이 연구   

## 2.4 Prediction

click history 가 주어지면 un-normalized된 multinomial probability f theta(z)를 통하여 모든 아이템에 rank 매김

# 4. EMPIRICAL STUDY

## 4.2 Metrics   
Recall@R(실제 True인 것 중에서 모델이 True라고 예측한 것의 비율/아이템이 좋은 추천인지 그렇지 않은지에만 관심), NDCG@R(normalized discounted cumulative gain, 제 랭킹의 위치,

즉 추천된 아이템이 얼마나 좋은지 혹은 어떠한 랭킹보다 더 좋은지 나쁜지에 더 중점) 두 개의 rank metric을 사용해 성능을 평가. 각 유저에 대해서 held-out 아이템에 대한 predicted rank와 true rank를 비교 [랭킹 기반 평가에 대하여](https://jyoondev.tistory.com/131)

## 4.3 Experimental setup
논문에서 보면 held-out (validation and test)의 경우, user들의 전체 click history가 아니라 모델이 필요한 user-level representation을 학습하기 위한 click history의 일부분만을 이용하여 평가

training과 evaluation에서 모두 user’s click history가 나타날 경우의 문제 이것 때문에 subsample을 하는 것 → held-out user들의 click history에서 80%를 랜덤하게 선택하여 fold-in set을 구성하여 모델이 user-level representation를 학습하게 하고 나머지 20%의 click history에 대해서 평가

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/DEEPLEARNING/ml-20m

/content/drive/MyDrive/DEEPLEARNING/ml-20m


# Data preprocessing

In [ ]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.3 MB 53 kB/s 
     |████████████████████████████████| 3.1 MB 32.5 MB/s 
     |████████████████████████████████| 488 kB 45.6 MB/s 
     |████████████████████████████████| 50 kB 4.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you

In [ ]:
import shutil
import sys

import numpy as np
from scipy import sparse

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sn
sn.set()

import pandas as pd

import tensorflow as tf
from tensorflow.contrib.layers import apply_regularization, l2_regularizer

import bottleneck as bn

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
#!pip install pytorch-lightning

In [ ]:
#mport os
#import sys 
#import pandas as pd
#import numpy as np
#from scipy import sparse

#import torch
#import pytorch_lightning as pl

In [ ]:
raw_data = pd.read_csv('ratings.csv', header=0)

In [ ]:
# binarize the data (only keep ratings >= 4) = binarize 0,1로 4이상이면 1
raw_data = raw_data[raw_data['rating'] > 3.5]

In [ ]:
raw_data.head()

,userId,movieId,rating,timestamp
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826
10,1,293,4.0,1112484703


In [ ]:
len(raw_data)

9995410

## Data splitting procedure   
* Select 10K users as heldout users, 10K users as validation users, and the rest of the users for training
* Use all the items from the training users as item set
* For each of both validation and test user, subsample 80% as fold-in data and the rest for prediction

In [ ]:
def filter_triplets(tp, min_uc=5, min_sc=0): # 아이템이나 유저를 거르는 과정
   """
   Args:
      tp ([DataFrame]): [Movielens Dataset]
      min_uc (int, optional): [users who clicked on at least min_uc items]. Defaults to 5.
      min_sc (int, optional): [items which were clicked on by at least min_sc users]. Defaults to 0.
   """
   if min_sc > 0:
      item_count = tp.groupby('movieId').size()
      tp = tp[tp['movieId'].isin(item_count.index[item_count >= min_sc])]
   if min_uc > 0:
      user_count = tp.groupby('userId').size()
      tp = tp[tp['userId'].isin(user_count.index[user_count >= min_uc])]

   user_count, item_count = tp.groupby(
         'userId').size(), tp.groupby('movieId').size()
   return tp, user_count, item_count

[pandas.DataFrame.isin](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html)

In [ ]:
def filtering_results(raw_data, user_activity, item_popularity):
    sparsity = 1. * raw_data.shape[0] / (user_activity.shape[0] * item_popularity.shape[0])
    print("After filtering, there are %d watching events from %d users and %d movies (sparsity: %.3f%%)" %
          (raw_data.shape[0], user_activity.shape[0], item_popularity.shape[0], sparsity * 100))


Only keep items that are clicked on by at least 5 users

In [ ]:
raw_data, user_activity, item_popularity = filter_triplets(raw_data)

In [ ]:
filtering_results(raw_data, user_activity, item_popularity)

After filtering, there are 9990682 watching events from 136677 users and 20720 movies (sparsity: 0.353%)


In [ ]:
# Shuffle User Index            
unique_uid = user_activity.index
idx_perm = np.random.permutation(len(unique_uid))
unique_uid = unique_uid[idx_perm]

In [ ]:
unique_uid

Int64Index([110837,  60556,  32680,  23154,  98115, 120768, 120124, 118469,
              9158,  47111,
            ...
             21366,  62873,  53705, 120853, 131391,  59098,  87268,  25798,
             17577,  23269],
           dtype='int64', name='userId', length=136677)

In [ ]:
def split_users(unique_uid, n_heldout_users=10000):
  # n_heldout_users=10000   
   """Split train / validation / test users.
      Select 10K users as heldout users, 10K users as validation users, and the rest of the users for training
   Args:
       unique_uid (Array): [randomly permutated User Index]
       n_heldout_users (int)
   """
   n_users = len(unique_uid)
   tr_users = unique_uid[:(n_users - n_heldout_users * 2)]
   vd_users = unique_uid[(n_users - n_heldout_users * 2): (n_users - n_heldout_users)]
   te_users = unique_uid[(n_users - n_heldout_users):]
   return tr_users, vd_users, te_users

In [ ]:
# Split Users
tr_users, vd_users, te_users = split_users(unique_uid)
      
train_plays = raw_data.loc[raw_data['userId'].isin(tr_users)]
vad_plays = raw_data.loc[raw_data['userId'].isin(vd_users)]
test_plays = raw_data.loc[raw_data['userId'].isin(te_users)]

In [ ]:
print(len(train_plays))
print(len(vad_plays))
print(len(test_plays))

8514295
743013
733374


In [ ]:
def split_train_test_proportion(data, test_prop=0.2):
   data_grouped_by_user = data.groupby('userId')
   tr_list, te_list = list(), list()
   
   for i, (_, group) in enumerate(data_grouped_by_user):
      n_items_u = len(group)  # per user, item count
      
      if n_items_u >= 5:
         idx = np.zeros(n_items_u, dtype='bool')
         samples = np.random.choice(n_items_u, size=int(
               test_prop * n_items_u), replace=False).astype('int64')
         idx[samples] = True

         tr_list.append(group[np.logical_not(idx)])
         te_list.append(group[idx])
      else:
         tr_list.append(group)

      if i % 1000 == 0:
            print("%d users sampled" % i)
            sys.stdout.flush()
            
   data_tr = pd.concat(tr_list)
   data_te = pd.concat(te_list)
   print('Done')
   return data_tr, data_te

In [ ]:
def numerize(tp, profile2id, show2id):
    uid = tp['userId'].apply(lambda x: profile2id[x])
    sid = tp['movieId'].apply(lambda x: show2id[x])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [ ]:
# Dictionarize the Unique UserId and MovieId
unique_sid = pd.unique(train_plays['movieId'])
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(unique_uid))

In [ ]:
 # Filtering Unique MovieId
vad_plays = vad_plays.loc[vad_plays['movieId'].isin(unique_sid)]
test_plays = test_plays.loc[test_plays['movieId'].isin(unique_sid)]

논문에서 보면 held-out (validation and test)의 경우, user들의 전체 click history가 아니라 모델이 필요한 **user-level representation을 학습하기 위한** click history의 일부분만을 이용하여 평가   

training과 evaluation에서 모두 user’s click history가 나타날 경우의 문제 이것 때문에 subsample을 하는 것
   → held-out user들의 click history에서 80%를 랜덤하게 선택하여 fold-in set을 구성하여 모델이 user-level representation를 학습하게 하고 나머지 20%의 click history에 대해서 평가

In [ ]:
 # Split Dataset ❓❓❓ why subsample?
vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)
test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)

0 users sampled
1000 users sampled
2000 users sampled
3000 users sampled
4000 users sampled
5000 users sampled
6000 users sampled
7000 users sampled
8000 users sampled
9000 users sampled
Done
0 users sampled
1000 users sampled
2000 users sampled
3000 users sampled
4000 users sampled
5000 users sampled
6000 users sampled
7000 users sampled
8000 users sampled
9000 users sampled
Done


## Save the data

In [ ]:
train_data = numerize(train_plays, profile2id, show2id)
train_data.to_csv('train.csv', index=False)

vad_data_tr = numerize(vad_plays_tr, profile2id, show2id)
vad_data_tr.to_csv('validation_tr.csv',index=False)

vad_data_te = numerize(vad_plays_te, profile2id, show2id)
vad_data_te.to_csv('validation_te.csv',index=False)

test_data_tr = numerize(test_plays_tr, profile2id, show2id)
test_data_tr.to_csv('test_tr.csv',index=False)

test_data_te = numerize(test_plays_te, profile2id, show2id)
test_data_te.to_csv('test_te.csv',index=False)

# Model definition and training

Generative process: For each user $u$, the model starts by sampling a $K$-dimensional latent representation $\mathbf{z}_u$ from a standard Gaussian prior. The latent representation $\mathbf{z}_u$ is transformed via a non-linear function $f_\theta (\cdot) \in \mathbb{R}^I$ to produce a probability distribution over $I$ items $\pi (\mathbf{z}_u)$ from which the click history $\mathbf{x}_u$ is assumed to have been drawn:

In [ ]:
#import os
#import logging
#import numpy as np

#import torch
#import torch.nn as nn
#import torch.nn.functional as F
#from torch.utils.tensorboard import SummaryWriter

#import pytorch_lightning as pl

#from models import MultiVAE
#from models import MultiDAE

In [ ]:
class MultiDAE(object):
    def __init__(self, p_dims, q_dims=None, lam=0.01, lr=1e-3, random_seed=None):
      # input = pdims, output = qdims
        self.p_dims = p_dims
        if q_dims is None:
            self.q_dims = p_dims[::-1]
        else:
            assert q_dims[0] == p_dims[-1], "Input and output dimension must equal each other for autoencoders."
            assert q_dims[-1] == p_dims[0], "Latent dimension for p- and q-network mismatches." # assert는 뒤의 조건이 True가 아니면 AssertError를 발생한다.
            self.q_dims = q_dims
        self.dims = self.q_dims + self.p_dims[1:]
        
        self.lam = lam
        self.lr = lr
        self.random_seed = random_seed

        self.construct_placeholders()

    def construct_placeholders(self):        
        self.input_ph = tf.placeholder(
            dtype=tf.float32, shape=[None, self.dims[0]])
        self.keep_prob_ph = tf.placeholder_with_default(1.0, shape=None)

    def build_graph(self):

        self.construct_weights()

        saver, logits = self.forward_pass()
        log_softmax_var = tf.nn.log_softmax(logits)

        # per-user average negative log-likelihood
        neg_ll = -tf.reduce_mean(tf.reduce_sum(
            log_softmax_var * self.input_ph, axis=1))
        # apply regularization to weights - 간단하게 가자
        reg = l2_regularizer(self.lam)
        reg_var = apply_regularization(reg, self.weights)
        # tensorflow l2 regularization multiply 0.5 to the l2 norm
        # multiply 2 so that it is back in the same scale
        loss = neg_ll + 2 * reg_var
        
        train_op = tf.train.AdamOptimizer(self.lr).minimize(loss)

        # add summary statistics
        tf.summary.scalar('negative_multi_ll', neg_ll)
        tf.summary.scalar('loss', loss)
        merged = tf.summary.merge_all()
        return saver, logits, loss, train_op, merged

    def forward_pass(self):
        # construct forward graph        
        h = tf.nn.l2_normalize(self.input_ph, 1)
        h = tf.nn.dropout(h, self.keep_prob_ph)
        
        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            h = tf.matmul(h, w) + b
            
            if i != len(self.weights) - 1:
                h = tf.nn.tanh(h)
        return tf.train.Saver(), h

    def construct_weights(self):

        self.weights = []
        self.biases = []
        
        # define weights
        for i, (d_in, d_out) in enumerate(zip(self.dims[:-1], self.dims[1:])):
            weight_key = "weight_{}to{}".format(i, i+1)
            bias_key = "bias_{}".format(i+1)
            
            self.weights.append(tf.get_variable(
                name=weight_key, shape=[d_in, d_out],
                initializer=tf.contrib.layers.xavier_initializer(
                    seed=self.random_seed)))
            
            self.biases.append(tf.get_variable(
                name=bias_key, shape=[d_out],
                initializer=tf.truncated_normal_initializer(
                    stddev=0.001, seed=self.random_seed)))
            
            # add summary stats
            tf.summary.histogram(weight_key, self.weights[-1])
            tf.summary.histogram(bias_key, self.biases[-1])


The objective of Multi-VAE^{PR} (evidence lower-bound, or ELBO) for a single user $u$ is:$$
\mathcal{L}_u(\theta, \phi) = \mathbb{E}_{q_\phi(z_u | x_u)}[\log p_\theta(x_u | z_u)] - \beta \cdot KL(q_\phi(z_u | x_u) \| p(z_u))
$$where $q_\phi$ is the approximating variational distribution (inference model). $\beta$ is the additional annealing parameter that we control. The objective of the entire dataset is the average over all the users. It can be trained almost the same as Multi-DAE, thanks to reparametrization trick.

In [ ]:
class MultiVAE(MultiDAE): # DAE의 요소들을 상속하는 VAE = DAE 기능을 사용할 수 있음

    def construct_placeholders(self):
        super(MultiVAE, self).construct_placeholders() # super : 자식에서 부모를 사용하고 싶을 때 사용

        # placeholders with default values when scoring
        self.is_training_ph = tf.placeholder_with_default(0., shape=None)
        self.anneal_ph = tf.placeholder_with_default(1., shape=None)
        
    def build_graph(self):
        self._construct_weights()

        saver, logits, KL = self.forward_pass()
        log_softmax_var = tf.nn.log_softmax(logits)

        neg_ll = -tf.reduce_mean(tf.reduce_sum( # loss function (reconstruction error) = 원 데이터에 대한 likelihood
            log_softmax_var * self.input_ph,
            axis=-1))
        # apply regularization to weights
        reg = l2_regularizer(self.lam)
        
        reg_var = apply_regularization(reg, self.weights_q + self.weights_p)
        # tensorflow l2 regularization multiply 0.5 to the l2 norm
        # multiply 2 so that it is back in the same scale
        neg_ELBO = neg_ll + self.anneal_ph * KL + 2 * reg_var # elbo식
        
        train_op = tf.train.AdamOptimizer(self.lr).minimize(neg_ELBO)

        # add summary statistics
        tf.summary.scalar('negative_multi_ll', neg_ll)
        tf.summary.scalar('KL', KL)
        tf.summary.scalar('neg_ELBO_train', neg_ELBO)
        merged = tf.summary.merge_all()

        return saver, logits, neg_ELBO, train_op, merged
    
    def q_graph(self):
        mu_q, std_q, KL = None, None, None
        
        h = tf.nn.l2_normalize(self.input_ph, 1)
        h = tf.nn.dropout(h, self.keep_prob_ph)
        
        for i, (w, b) in enumerate(zip(self.weights_q, self.biases_q)):
            h = tf.matmul(h, w) + b
            
            if i != len(self.weights_q) - 1: # 이전까지 activation 적용
                h = tf.nn.tanh(h)
            else: # 다 돌고나면 KL 계산
                mu_q = h[:, :self.q_dims[-1]]
                logvar_q = h[:, self.q_dims[-1]:]

                std_q = tf.exp(0.5 * logvar_q)
                KL = tf.reduce_mean(tf.reduce_sum(
                        0.5 * (-logvar_q + tf.exp(logvar_q) + mu_q**2 - 1), axis=1))
        return mu_q, std_q, KL

    def p_graph(self, z):
        h = z
        
        for i, (w, b) in enumerate(zip(self.weights_p, self.biases_p)):
            h = tf.matmul(h, w) + b
            
            if i != len(self.weights_p) - 1:
                h = tf.nn.tanh(h)
        return h

    def forward_pass(self):
        # q-network
        mu_q, std_q, KL = self.q_graph()
        epsilon = tf.random_normal(tf.shape(std_q))

        sampled_z = mu_q + self.is_training_ph *\
            epsilon * std_q

        # p-network
        logits = self.p_graph(sampled_z)
        
        return tf.train.Saver(), logits, KL

    def _construct_weights(self):
        self.weights_q, self.biases_q = [], []
        
        for i, (d_in, d_out) in enumerate(zip(self.q_dims[:-1], self.q_dims[1:])):
            if i == len(self.q_dims[:-1]) - 1:
                # we need two sets of parameters for mean and variance,
                # respectively
                d_out *= 2
            weight_key = "weight_q_{}to{}".format(i, i+1)
            bias_key = "bias_q_{}".format(i+1)
            
            self.weights_q.append(tf.get_variable(
                name=weight_key, shape=[d_in, d_out],
                initializer=tf.contrib.layers.xavier_initializer(
                    seed=self.random_seed)))
            
            self.biases_q.append(tf.get_variable(
                name=bias_key, shape=[d_out],
                initializer=tf.truncated_normal_initializer(
                    stddev=0.001, seed=self.random_seed)))
            
            # add summary stats
            tf.summary.histogram(weight_key, self.weights_q[-1])
            tf.summary.histogram(bias_key, self.biases_q[-1])
            
        self.weights_p, self.biases_p = [], []

        for i, (d_in, d_out) in enumerate(zip(self.p_dims[:-1], self.p_dims[1:])):
            weight_key = "weight_p_{}to{}".format(i, i+1)
            bias_key = "bias_p_{}".format(i+1)
            self.weights_p.append(tf.get_variable(
                name=weight_key, shape=[d_in, d_out],
                initializer=tf.contrib.layers.xavier_initializer(
                    seed=self.random_seed)))
            
            self.biases_p.append(tf.get_variable(
                name=bias_key, shape=[d_out],
                initializer=tf.truncated_normal_initializer(
                    stddev=0.001, seed=self.random_seed)))
            
            # add summary stats
            tf.summary.histogram(weight_key, self.weights_p[-1])
            tf.summary.histogram(bias_key, self.biases_p[-1])

In [ ]:
#import numpy as np 
import bottleneck as bn

def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=100):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG


def Recall_at_k_batch(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

# Training/validation data, hyperparameters

In [ ]:
n_items = len(unique_sid)

In [ ]:
def load_train_data(csv_file):
    tp = pd.read_csv(csv_file)
    n_users = tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows), #sparse matrix로 변환해서 불러오기
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data

In [ ]:
train_data = load_train_data('train.csv')

In [ ]:
def load_tr_te_data(csv_file_tr, csv_file_te):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te

In [ ]:
vad_data_tr, vad_data_te = load_tr_te_data( 'validation_tr.csv',
                                           'validation_te.csv')

Set up training hyperparameters

In [ ]:
N = train_data.shape[0]
idxlist = range(N)

# training batch size
batch_size = 500
batches_per_epoch = int(np.ceil(float(N) / batch_size))

N_vad = vad_data_tr.shape[0]
idxlist_vad = range(N_vad)

# validation batch size (since the entire validation set might not fit into GPU memory)
batch_size_vad = 2000

# the total number of gradient updates for annealing
total_anneal_steps = 200000
# largest annealing parameter
anneal_cap = 0.2

Evaluate function: Normalized discounted cumulative gain (NDCG@k) and Recall@k

# Train a Multi-VAE   
For ML-20M dataset, we set both the generative function $f_\theta(\cdot)$ and the inference model $g_\phi(\cdot)$ to be 3-layer multilayer perceptron (MLP) with symmetrical architecture. 

The generative function is a [200 -> 600 -> n_items] MLP, which means the inference function is a [n_items -> 600 -> 200] MLP. Thus the overall architecture for the Multi-VAE^{PR} is [n_items -> 600 -> 200 -> 600 -> n_items].

In [ ]:
p_dims = [200, 600, n_items]

In [ ]:
tf.reset_default_graph()
vae = MultiVAE(p_dims, lam=0.0, random_seed=98765)

saver, logits_var, loss_var, train_op_var, merged_var = vae.build_graph()

ndcg_var = tf.Variable(0.0)
ndcg_dist_var = tf.placeholder(dtype=tf.float64, shape=None)
ndcg_summary = tf.summary.scalar('ndcg_at_k_validation', ndcg_var)
ndcg_dist_summary = tf.summary.histogram('ndcg_at_k_hist_validation', ndcg_dist_var)
merged_valid = tf.summary.merge([ndcg_summary, ndcg_dist_summary])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
INFO:tensorflow:Scale of 0 disables regularizer.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
arch_str = "I-%s-I" % ('-'.join([str(d) for d in vae.dims[1:-1]]))

In [ ]:
import os
log_dir = '/volmount/log/ml-20m/VAE_anneal{}K_cap{:1.1E}/{}'.format(
    total_anneal_steps/1000, anneal_cap, arch_str)

if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

print("log directory: %s" % log_dir)
summary_writer = tf.summary.FileWriter(log_dir, graph=tf.get_default_graph())

log directory: /volmount/log/ml-20m/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I


In [ ]:
chkpt_dir = '/volmount/chkpt/ml-20m/VAE_anneal{}K_cap{:1.1E}/{}'.format(
    total_anneal_steps/1000, anneal_cap, arch_str)

if not os.path.isdir(chkpt_dir):
    os.makedirs(chkpt_dir) 
    
print("chkpt directory: %s" % chkpt_dir)

chkpt directory: /volmount/chkpt/ml-20m/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I


In [ ]:
n_epochs=200

In [ ]:
ndcgs_vad = []

with tf.Session() as sess:

    init = tf.global_variables_initializer()
    sess.run(init)

    best_ndcg = -np.inf

    update_count = 0.0
    
    for epoch in range(n_epochs):
        np.random.shuffle(list(idxlist))
        # train for one epoch
        for bnum, st_idx in enumerate(range(0, N, batch_size)):
            end_idx = min(st_idx + batch_size, N)
            X = train_data[idxlist[st_idx:end_idx]]
            
            if sparse.isspmatrix(X):
                X = X.toarray()
            X = X.astype('float32')           
            
            if total_anneal_steps > 0:
                anneal = min(anneal_cap, 1. * update_count / total_anneal_steps)
            else:
                anneal = anneal_cap
            
            feed_dict = {vae.input_ph: X, 
                         vae.keep_prob_ph: 0.5, 
                         vae.anneal_ph: anneal,
                         vae.is_training_ph: 1}        
            sess.run(train_op_var, feed_dict=feed_dict)

            if bnum % 100 == 0:
                summary_train = sess.run(merged_var, feed_dict=feed_dict)
                summary_writer.add_summary(summary_train, 
                                           global_step=epoch * batches_per_epoch + bnum) 
            
            update_count += 1
        
        # compute validation NDCG
        ndcg_dist = []
        for bnum, st_idx in enumerate(range(0, N_vad, batch_size_vad)):
            end_idx = min(st_idx + batch_size_vad, N_vad)
            X = vad_data_tr[idxlist_vad[st_idx:end_idx]]

            if sparse.isspmatrix(X):
                X = X.toarray()
            X = X.astype('float32')
        
            pred_val = sess.run(logits_var, feed_dict={vae.input_ph: X} )
            # exclude examples from training and validation (if any)
            pred_val[X.nonzero()] = -np.inf
            ndcg_dist.append(NDCG_binary_at_k_batch(pred_val, vad_data_te[idxlist_vad[st_idx:end_idx]]))
        
        ndcg_dist = np.concatenate(ndcg_dist)
        ndcg_ = ndcg_dist.mean()
        ndcgs_vad.append(ndcg_)
        merged_valid_val = sess.run(merged_valid, feed_dict={ndcg_var: ndcg_, ndcg_dist_var: ndcg_dist})
        summary_writer.add_summary(merged_valid_val, epoch)

        # update the best model (if necessary)
        if ndcg_ > best_ndcg:
            saver.save(sess, '{}/model'.format(chkpt_dir))
            best_ndcg = ndcg_

KeyboardInterrupt: ignored

In [ ]:
plt.figure(figsize=(12, 3))
plt.plot(ndcgs_vad)
plt.ylabel("Validation NDCG@100")
plt.xlabel("Epochs")
pass

## Load the test data and compute test metrics

In [ ]:
test_data_tr, test_data_te = load_tr_te_data(
    'test_tr.csv','test_te.csv')

In [ ]:
N_test = test_data_tr.shape[0]
idxlist_test = range(N_test)

batch_size_test = 2000

In [ ]:
tf.reset_default_graph()
vae = MultiVAE(p_dims, lam=0.0)
saver, logits_var, _, _, _ = vae.build_graph()

Load the best performing model on the validation set

In [ ]:
chkpt_dir = '/volmount/chkpt/ml-20m/VAE_anneal{}K_cap{:1.1E}/{}'.format(
    total_anneal_steps/1000, anneal_cap, arch_str)
print("chkpt directory: %s" % chkpt_dir)

In [ ]:
n100_list, r20_list, r50_list = [], [], []

with tf.Session() as sess:
    saver.restore(sess, '{}/model'.format(chkpt_dir))

    for bnum, st_idx in enumerate(range(0, N_test, batch_size_test)):
        end_idx = min(st_idx + batch_size_test, N_test)
        X = test_data_tr[idxlist_test[st_idx:end_idx]]

        if sparse.isspmatrix(X):
            X = X.toarray()
        X = X.astype('float32')

        pred_val = sess.run(logits_var, feed_dict={vae.input_ph: X})
        # exclude examples from training and validation (if any)
        pred_val[X.nonzero()] = -np.inf
        n100_list.append(NDCG_binary_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=100))
        r20_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=20))
        r50_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=50))
    
n100_list = np.concatenate(n100_list)
r20_list = np.concatenate(r20_list)
r50_list = np.concatenate(r50_list)

In [ ]:
print("Test NDCG@100=%.5f (%.5f)" % (np.mean(n100_list), np.std(n100_list) / np.sqrt(len(n100_list))))
print("Test Recall@20=%.5f (%.5f)" % (np.mean(r20_list), np.std(r20_list) / np.sqrt(len(r20_list))))
print("Test Recall@50=%.5f (%.5f)" % (np.mean(r50_list), np.std(r50_list) / np.sqrt(len(r50_list))))